In [1]:
import numpy as np
import pandas as pd
import os
import librosa
import wave
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import *
from sklearn.preprocessing import LabelEncoder
# from keras.optimizer_v1 import RMSprop



Run till


In [2]:
ravdess_df = pd.read_csv('D:\Final Project\Speech\combined.csv')
ravdess_df = ravdess_df.iloc[:,1:]
ravdess_df

,gender,emotion,actor,path
0,female,fear,1,/content/drive/MyDrive/TESS Toronto emotional ...
1,female,fear,1,/content/drive/MyDrive/TESS Toronto emotional ...
2,female,fear,1,/content/drive/MyDrive/TESS Toronto emotional ...
3,female,fear,1,/content/drive/MyDrive/TESS Toronto emotional ...
4,female,fear,1,/content/drive/MyDrive/TESS Toronto emotional ...
...,...,...,...,...
4235,female,fear,24,/content/drive/MyDrive/audio/Actor_24/03-01-06...
4236,female,fear,24,/content/drive/MyDrive/audio/Actor_24/03-01-06...
4237,female,angry,24,/content/drive/MyDrive/audio/Actor_24/03-01-05...
4238,female,surprise,24,/content/drive/MyDrive/audio/Actor_24/03-01-08...


In [3]:
def extract_mfcc(wave_file_name):
  y,sr = librosa.load(wave_file_name)
  # S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,fmax=8000)
  mfcc=librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40)
  mean_mfcc = np.mean(mfcc.T,axis=0)
  # mean_mfcc=np.hstack((mfcc,emo))
  return mean_mfcc

In [4]:
y, sr = librosa.load(r'D:\Final Project\Speech\audio\Actor_01\03-01-01-01-02-02-01.wav')
# # S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128,fmax=8000)
# mfcc=librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40)
# # mean_mfcc = np.mean(mfcc,axis=1)
# mfcc[0]
emo = np.full(shape=(25, 1), fill_value=1, dtype=int)  # corrected the dtype
mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=25)
mean_mfcc = np.hstack((mfcc, emo))
mean_mfcc

array([[-8.38642456e+02, -8.37072998e+02, -8.34627991e+02, ...,
        -8.42745422e+02, -8.45989136e+02,  1.00000000e+00],
       [ 6.96634865e+00,  7.81747055e+00,  1.02320747e+01, ...,
         4.99251461e+00,  4.18295473e-01,  1.00000000e+00],
       [ 1.29587030e+00, -1.36998296e-03,  1.34730077e+00, ...,
         4.95246315e+00,  4.17444944e-01,  1.00000000e+00],
       ...,
       [ 3.21494865e+00,  3.81341934e+00,  1.75828004e+00, ...,
         1.42855537e+00,  2.88627476e-01,  1.00000000e+00],
       [ 3.40211010e+00,  2.80362749e+00,  2.05245447e+00, ...,
         1.29192448e+00,  2.77273625e-01,  1.00000000e+00],
       [ 1.52598834e+00,  9.44242179e-02, -1.37573528e+00, ...,
         1.16945744e+00,  2.65543997e-01,  1.00000000e+00]])

In [5]:
from IPython.display import Audio
y,sr = librosa.load(r'D:\Final Project\Speech\audio\Actor_01\03-01-01-01-02-02-01.wav')
Audio(data=y,rate=sr)

In [6]:
# TRAIN TEST SPLIT DATA
train,test = train_test_split(ravdess_df, test_size=0.2, random_state=0,
                               stratify=ravdess_df[['emotion','gender','actor']])

In [7]:
X_train = train.iloc[:, 3:]
y_train = train.iloc[:,:2].drop(columns=['gender'])
print(X_train.shape)
print(test)

(3392, 1)
      gender  emotion  actor  \
2300  female  neutral      1   
1908  female     fear      1   
2277  female  neutral      1   
1143  female    happy      1   
577   female      sad      1   
...      ...      ...    ...   
1139  female    happy      1   
195   female     fear      1   
3144  female  neutral      6   
1570  female    angry      1   
3208    male    happy      7   

                                                   path  
2300  /content/drive/MyDrive/TESS Toronto emotional ...  
1908  /content/drive/MyDrive/TESS Toronto emotional ...  
2277  /content/drive/MyDrive/TESS Toronto emotional ...  
1143  /content/drive/MyDrive/TESS Toronto emotional ...  
577   /content/drive/MyDrive/TESS Toronto emotional ...  
...                                                 ...  
1139  /content/drive/MyDrive/TESS Toronto emotional ...  
195   /content/drive/MyDrive/TESS Toronto emotional ...  
3144  /content/drive/MyDrive/audio/Actor_06/03-01-01...  
1570  /content/drive/MyDr

In [8]:
#df = pd.DataFrame(columns=['log_spec'])
#counter=0

#for index,path in enumerate(train.path[:200]):
#  mfcc = extract_mfcc(path)
#  df.loc[counter] = [mfcc]
#  counter += 1
#  print(counter)
#print(len(df))
df =pd.read_csv(r'D:\Final Project\Speech\mfcc.csv')
df = df.iloc[: , 1:]
df

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
0,-507.20383,32.656650,-14.149189,-2.398801,-6.611685,-4.475557,-11.523510,-7.914782,-11.853230,-4.217081,...,3.141099,-1.414575,-0.608415,-2.243408,1.668388,0.557390,0.738123,0.407085,2.972007,surprise
1,-424.91150,72.836365,13.582559,-14.813837,-15.483583,6.020358,-11.732384,5.955402,-17.852468,-5.906166,...,0.386430,-3.698372,-2.359355,0.525397,2.094145,-4.072193,0.831913,-4.101292,0.082907,fear
2,-662.95710,41.473816,-18.418589,3.318224,-12.977219,-7.336224,-11.320430,-14.533319,-9.974374,-5.557361,...,1.424337,0.317421,3.119349,0.068752,1.490845,-1.150846,1.657690,0.109153,0.849677,surprise
3,-611.10657,79.380600,2.824459,16.471415,10.420895,-4.252988,-0.332738,-4.867650,-8.821871,1.940849,...,-1.887397,-3.486242,-1.849411,-3.482877,-0.303797,-3.329178,-2.793863,-4.038559,-2.321982,calm
4,-540.01166,51.239983,-22.972181,10.175825,1.167942,-8.053654,-1.710532,-9.832929,-2.802476,-1.464126,...,3.146587,3.634718,8.796452,6.198578,5.092316,2.126676,3.177321,3.404370,5.235992,happy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,-502.05188,43.859900,-2.604773,1.126535,-5.614835,-0.265754,-15.036654,-13.864769,-9.901254,-4.028796,...,1.853130,-1.086178,0.423127,-0.418280,3.166987,0.212238,0.325814,-1.025998,-0.498045,disgust
3388,-729.59503,70.421370,6.720570,23.453606,4.594159,6.021837,-2.202099,3.294228,0.536756,6.440284,...,1.657300,-0.365339,0.584623,0.511120,1.652351,-3.105796,-2.382192,-2.994608,-0.887968,calm
3389,-667.15466,92.273660,3.108789,20.048185,9.216145,-0.450933,-12.057720,-9.685620,1.276145,2.078562,...,-0.381134,-2.543870,-0.217840,-3.859147,-4.564312,-7.368496,-1.820121,-3.363920,-0.662408,calm
3390,-347.64536,54.039684,-15.523150,13.062150,-6.169755,3.703157,-6.078752,-11.714536,-13.542357,5.625887,...,6.362086,-2.329728,-2.529495,-9.231550,0.290030,1.129973,-1.206243,-4.214062,3.731706,fear


In [9]:
# EXTRACT EMOTION LABELS FROM TRAIN
labels = pd.DataFrame(train.emotion)
labels.reset_index(drop=True, inplace=True)
labels

,emotion
0,surprise
1,fear
2,surprise
3,calm
4,happy
...,...
3387,disgust
3388,calm
3389,calm
3390,fear


In [10]:
# ADD CORRESPONDING EMOTION LABELS TO DF['LOG_SPEC']
# type(df['log_spec'])
# mfcc_df = pd.DataFrame(df['log_spec'].values.tolist())
# mfcc = pd.concat([mfcc_df,labels], axis=1,ignore_index=True)
# mfcc = mfcc.rename(index=str, columns={40: "emotion"})
# mfcc = mfcc.iloc[:,:109]
mfcc = pd.read_csv(r'D:\Final Project\Speech\mfcc.csv')
mfcc = mfcc.iloc[:,1:]
print(mfcc)

              0          1          2          3          4         5  \
0    -507.20383  32.656650 -14.149189  -2.398801  -6.611685 -4.475557   
1    -424.91150  72.836365  13.582559 -14.813837 -15.483583  6.020358   
2    -662.95710  41.473816 -18.418589   3.318224 -12.977219 -7.336224   
3    -611.10657  79.380600   2.824459  16.471415  10.420895 -4.252988   
4    -540.01166  51.239983 -22.972181  10.175825   1.167942 -8.053654   
...         ...        ...        ...        ...        ...       ...   
3387 -502.05188  43.859900  -2.604773   1.126535  -5.614835 -0.265754   
3388 -729.59503  70.421370   6.720570  23.453606   4.594159  6.021837   
3389 -667.15466  92.273660   3.108789  20.048185   9.216145 -0.450933   
3390 -347.64536  54.039684 -15.523150  13.062150  -6.169755  3.703157   
3391 -295.21622  30.571106 -14.999253  29.672844  -7.821387 -7.727045   

              6          7          8         9  ...        31        32  \
0    -11.523510  -7.914782 -11.853230 -4.217081

In [11]:
combined_df = pd.concat([mfcc], ignore_index=True)
combined_df = combined_df.fillna(0)
combined_df.tail()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,emotion
3387,-502.05188,43.859900,-2.604773,1.126535,-5.614835,-0.265754,-15.036654,-13.864769,-9.901254,-4.028796,...,1.853130,-1.086178,0.423127,-0.418280,3.166987,0.212238,0.325814,-1.025998,-0.498045,disgust
3388,-729.59503,70.421370,6.720570,23.453606,4.594159,6.021837,-2.202099,3.294228,0.536756,6.440284,...,1.657300,-0.365339,0.584623,0.511120,1.652351,-3.105796,-2.382192,-2.994608,-0.887968,calm
3389,-667.15466,92.273660,3.108789,20.048185,9.216145,-0.450933,-12.057720,-9.685620,1.276145,2.078562,...,-0.381134,-2.543870,-0.217840,-3.859147,-4.564312,-7.368496,-1.820121,-3.363920,-0.662408,calm
3390,-347.64536,54.039684,-15.523150,13.062150,-6.169755,3.703157,-6.078752,-11.714536,-13.542357,5.625887,...,6.362086,-2.329728,-2.529495,-9.231550,0.290030,1.129973,-1.206243,-4.214062,3.731706,fear
3391,-295.21622,30.571106,-14.999253,29.672844,-7.821387,-7.727045,-8.047218,-9.664738,-4.616383,-2.845304,...,2.911048,-3.477996,3.098792,1.807947,7.685647,0.705828,1.244122,-4.595193,-2.219147,happy


In [12]:
X_train = combined_df.iloc[:, :-1]
y_train = combined_df.iloc[:, -1:]

In [13]:
# test = test.loc[test.index.repeat(25)].reset_index(drop=True)
test.reset_index(drop=True,inplace=True)


In [14]:
# df_combined = pd.concat([test,pd.DataFrame(df['mel_spectrogram'].values.tolist())],axis=1)
# df_combined = df_combined.fillna(0)
df_combined = pd.read_csv(r'D:\Final Project\Speech\mfcc_test.csv')
df_combined = df_combined.iloc[:,1:]

In [15]:
# DROP PATH COLUMN FOR MODELING
df_combined.drop(columns='path',inplace=True)

In [16]:
# CHECK TOP 5 ROWS
print(df_combined.shape)
df_combined.head()

(848, 43)


,gender,emotion,actor,0,1,2,3,4,5,6,...,30,31,32,33,34,35,36,37,38,39
0,female,neutral,1,-423.85110,72.622010,5.002045,34.543220,2.988348,-6.102084,-7.146608,...,-10.017267,1.003834,0.785221,15.430670,23.043825,32.179350,25.739586,15.907831,3.179427,-1.214063
1,female,fear,1,-275.35068,48.786465,-23.029780,23.829979,2.007111,-13.883335,-11.679904,...,-3.491931,3.237022,-0.913629,0.011729,-6.144578,1.180668,0.865935,-0.295077,-0.515710,3.326320
2,female,neutral,1,-462.33862,79.389380,28.032906,46.177563,-4.787593,0.363241,2.618909,...,-6.901717,0.561534,0.330665,10.256723,16.561417,29.265001,21.282614,19.126707,5.411518,1.594192
3,female,happy,1,-436.39105,64.843220,7.770212,-4.279943,-9.480633,7.624158,-16.362133,...,-0.448711,6.618950,-0.563480,3.798247,0.387800,6.184574,0.441807,4.829961,6.215880,11.550493
4,female,sad,1,-540.85640,80.595900,23.293245,18.689896,18.706978,14.204223,-3.318747,...,7.220472,12.138967,2.937183,5.546771,6.373443,10.849830,11.667699,12.886155,8.057577,7.176181


In [17]:
# SEPARATE DF_COMBINED INTO X_TEST AND Y_TEST
X_test = df_combined.iloc[:, 3:]
y_test = df_combined.iloc[:,:2].drop(columns=['gender'])
# X_test.loc[0] = [-4.1287537e+02,  5.0520481e+01, -8.6483097e+00, -4.7108097e+00,
#        -1.2350923e+01,  5.4682279e+00, -1.9826475e+01, -2.2179692e+00,
#        -1.3606742e+01, -4.2214799e+00, -9.3492384e+00,  8.0879450e+00,
#        -7.9176288e+00,  1.1303248e+00, -3.7942588e+00,  1.8726997e+00,
#        -6.2138438e+00, -5.5670347e+00, -1.1563614e+01, -8.4239130e+00,
#         6.4517802e-01,  8.0211363e+00,  1.0356618e+01,  2.6925550e+01,
#         2.4552359e+01,  2.2129726e+01,  1.5090271e+01,  1.0901266e+01,
#         2.5204284e+00,  1.0641685e+01,  9.5021925e+00,  1.2646760e+01,
#         5.0759921e+00, -4.0321288e-01,  3.1849222e+00,  2.0790806e+00,
#        -2.0313420e+00,  3.7414479e+00, -3.1421704e+00,  6.7317683e-01]
print(X_test)

             0          1          2          3          4          5  \
0   -423.85110  72.622010   5.002045  34.543220   2.988348  -6.102084   
1   -275.35068  48.786465 -23.029780  23.829979   2.007111 -13.883335   
2   -462.33862  79.389380  28.032906  46.177563  -4.787593   0.363241   
3   -436.39105  64.843220   7.770212  -4.279943  -9.480633   7.624158   
4   -540.85640  80.595900  23.293245  18.689896  18.706978  14.204223   
..         ...        ...        ...        ...        ...        ...   
843 -445.29880  93.780860  16.292828  -8.900312  -1.940576  -0.654583   
844 -430.77396  72.178080  12.635900 -10.016851  -8.728482   3.783648   
845 -705.46200  52.509598 -12.874601  11.076167  -1.324863  -7.192864   
846 -323.70233  38.732285 -17.726604   8.172358 -30.097532  -2.698441   
847 -614.27765  73.949090 -10.266552  16.662582   6.028456  -0.473712   

             6          7          8         9  ...         30         31  \
0    -7.146608 -12.150572  -5.767888  3.613931

In [18]:
# NORMALIZE DATA
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean)/std
X_test = (X_test - mean)/std
np.save('D:\Final Project\Speech\mean.npy', mean)
np.save('D:\Final Project\Speech\std.npy', std)
#mean = load('/content/drive/MyDrive/mean.npy')
#std = load('/content/drive/MyDrive/std.npy')

In [19]:
# TURN INTO ARRAYS
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [20]:
# ONE HOT ENCODE EMOTION LABELS
lb = LabelEncoder()
y_train = to_categorical(lb.fit_transform(y_train))
y_test = to_categorical(lb.fit_transform(y_test))
print(y_test[0:10])

[[0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


C:\Users\alang\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\alang\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


change made

In [21]:
print(lb.classes_)

['angry' 'calm' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']


here

In [22]:
# RESHAPE TO ADD 3D TENSOR FOR 1D CNN INPUT
X_train = X_train[:,:,np.newaxis]
X_test = X_test[:,:,np.newaxis]
X_test.shape

(848, 40, 1)

In [23]:
X_test

array([[[ 3.89391129e-01],
        [ 7.03459343e-01],
        [ 3.10691599e-01],
        ...,
        [ 1.28259507e+00],
        [ 8.59477184e-02],
        [-1.02791815e+00]],

       [[ 1.66008856e+00],
        [-4.41044435e-01],
        [-1.47489464e+00],
        ...,
        [-7.65467492e-01],
        [-4.79448835e-01],
        [-1.38415617e-01]],

       [[ 6.00587744e-02],
        [ 1.02840599e+00],
        [ 1.77772375e+00],
        ...,
        [ 1.68946398e+00],
        [ 4.27482122e-01],
        [-4.77755440e-01]],

       ...,

       [[-2.02031417e+00],
        [-2.62271942e-01],
        [-8.28024597e-01],
        ...,
        [-4.33514776e-01],
        [ 1.58925612e-01],
        [ 4.85160818e-01]],

       [[ 1.24635022e+00],
        [-9.23812795e-01],
        [-1.13709009e+00],
        ...,
        [ 1.49337760e-03],
        [-2.87181539e-01],
        [-5.86005022e-01]],

       [[-1.24006237e+00],
        [ 7.67181321e-01],
        [-6.61895692e-01],
        ...,
        

In [24]:
from numpy import load, save

# Save arrays to files
save('D:\Final Project\Speech\X_train.npy', X_train)
save('D:\Final Project\Speech\Y_train.npy', y_train)
save('D:\Final Project\Speech\X_test.npy', X_test)
save('D:\Final Project\Speech\Y_test.npy', y_test)

# Load arrays from files
X_train = load('D:\Final Project\Speech\X_train.npy')
y_train = load('D:\Final Project\Speech\Y_train.npy')
X_test = load('D:\Final Project\Speech\X_test.npy')
y_test = load('D:\Final Project\Speech\Y_test.npy')


In [25]:
# # BUILD 1D CNN LAYERS
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.regularizers import l2  # Import l2 regularizer
# import tensorflow as tf

# model = Sequential()
# model.add(Conv1D(32, 8, activation='relu', padding='same', input_shape=(X_train.shape[1], 1)))
# model.add(Conv1D(128, 8, activation='relu', padding='same', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Dropout(0.25))

# model.add(Conv1D(128, 8, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=(2)))
# model.add(Dropout(0.25))
# model.add(Flatten())

# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(8, activation='softmax'))

# model.summary()

# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=0.0001,
#     decay_steps=10000,
#     decay_rate=1e-6)
# opt = tf.keras.optimizers.Adam(lr_schedule)


not this

In [26]:
# from keras.callbacks import ModelCheckpoint
# checkpoint = ModelCheckpoint('face_detector.h15', monitor= 'val_loss', mode= 'min', save_best_only = True, verbose= 1)

# # FIT MODEL
# model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
# model_history=model.fit(X_train, y_train,batch_size=16, epochs=20, validation_data=(X_test, y_test),callbacks=checkpoint)


not this

In [27]:
# model.save_weights(r'D:\Final Project\Speech\best_model_weights.h5')
# model.save(r'D:\Final Project\Speech\best_model.h5')
# print("Saved model to disk")
# from tensorflow.keras.layers import Conv2D, MaxPooling2D
# from tensorflow.keras.regularizers import l2

# # # BUILD 1D CNN LAYERS
# model = keras.models.load_model(r'D:\Final Project\Speech\best_model.h5')


not this


In [28]:
# # PRINT MODEL HISTORY OF ACCURACY AND LOSS OVER EPOCHS
# plt.plot(model_history.history['accuracy'])
# plt.plot(model_history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.savefig('Augmented_Model_Accuracy.png')
# plt.show()
# # summarize history for loss
# plt.plot(model_history.history['loss'])
# plt.plot(model_history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.savefig('Augmented_Model_Loss.png')
# plt.show()

In [29]:
# # PREDICTION LABELS
# # model = tf.keras.models.load_model('/home/adarsh/Downloads/best_model.h5')
# # model.load_weights('/home/adarsh/Downloads/best_model_weights.h5')
# predictions = model.predict(X_test)
# predictions=predictions.argmax(axis=1)
# predictions
# predictions = predictions.astype(int).flatten()
# predictions = (lb.inverse_transform((predictions)))
# predictions = pd.DataFrame({'Predicted Values': predictions})
# # ACTUAL LABELS
# actual=y_test.argmax(axis=1)
# actual = actual.astype(int).flatten()
# actual = (lb.inverse_transform((actual)))
# actual = pd.DataFrame({'Actual Values': actual})
# predictions
# # # COMBINE PREDICTION AND ACTUAL LABELS
# pred = pd.concat([predictions,actual],axis=1,ignore_index=True)
# pred
# # # # predictions
# # # actual

In [30]:
# # CREATE CONFUSION MATRIX OF ACTUAL VS. PREDICTION
# from sklearn.metrics import confusion_matrix
# import seaborn as sns

# cm = confusion_matrix(actual, predictions)
# plt.figure(figsize = (12, 10))
# cm = pd.DataFrame(cm , index = [i for i in lb.classes_] , columns = [i for i in lb.classes_])
# ax = sns.heatmap(cm, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')
# bottom, top = ax.get_ylim()
# ax.set_ylim(bottom + 0.5, top - 0.5)
# plt.title('Confusion Matrix', size=20)
# plt.xlabel('Predicted Labels', size=14)
# plt.ylabel('Actual Labels', size=14)
# plt.savefig('Augmented_Model_Confusion_Matrix.png')
# plt.show()

In [31]:
# from sklearn.metrics import classification_report
# print(classification_report(actual, predictions, target_names = ['angry','calm','disgust','fear','happy','neutral','sad','surprise']))

In [32]:
# from scipy.io.wavfile import write
# import sounddevice as sd
# import os

# # Set the directory to save the audio file
# output_dir = 'D:/Final Project/Speech/'


# # Define the duration and sampling rate for recording
# duration = 6  # Duration in seconds
# fs = 44100  # Sampling rate in Hz

# # Record audio
# print("Recording...")
# audio_data = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
# sd.wait()  # Wait for recording to complete
# print("Recording done.")

# # Save the audio file
# output_file = os.path.join(output_dir, 'recorded_audio.wav')
# write(output_file, fs, audio_data)

# print(f"Audio file saved at: {output_file}")


In [33]:
mean = np.load('D:\Final Project\Speech\mean.npy')
std = np.load('D:\Final Project\Speech\std.npy')
model = keras.models.load_model(r'D:\Final Project\Speech\best_model.h5')

In [34]:
import pyaudio
import wave
import time
import librosa
import numpy as np
from tensorflow.keras.models import load_model

# Define constants
FORMAT = pyaudio.paInt16  # Audio format
CHANNELS = 1  # Number of audio channels
RATE = 44100  # Sampling rate
CHUNK = 1024  # Audio chunk size
RECORD_SECONDS = 10  # Duration of each audio recording in seconds
FILE_NAME = 'audio.wav'  # Output WAV file name

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Open stream
stream = audio.open(format=FORMAT, channels=CHANNELS,
                     rate=RATE, input=True,
                     frames_per_buffer=CHUNK)

# Load the emotion classification model
model = load_model(r'D:\Final Project\Speech\best_model.h5')

try:
    while True:
        print("Recording...")
        frames = []
        
        # Record audio for RECORD_SECONDS
        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)
        
        # Save the recorded audio to a WAV file
        wf = wave.open(FILE_NAME, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        wf.close()
        
        print("Recording saved as 'audio.wav'")
        
        # Sleep for 10 seconds
        time.sleep(10)
        
        
        test_aud = extract_mfcc(r'D:\Final Project\Text\Final Output\audio.wav')
        test_aud
        #mean = np.mean(test_aud,axis=0)
        #std = np.std(test_aud,axis=0)
        test_auddf = (test_aud-mean)/std
        test_auddf = np.array([test_auddf])
        test_auddf
        test_auddf = test_auddf[:,:,np.newaxis]
        test_auddf
        # livedf2= pd.DataFrame(data=test_auddf)
        # livedf2 = livedf2.stack().to_frame().T
        # twodim= np.expand_dims(livedf2, axis=2)
        
        # Load the saved audio and extract MFCC features
        #y, sr = librosa.load(FILE_NAME)
        #mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        
        # Transpose MFCC array to have shape (time_steps, n_mfcc)
        #mfcc = mfcc.T
        
        # Expand dimensions to match the model's input shape
        #test_auddf = np.expand_dims(mfcc, axis=0)
        #test_auddf = np.expand_dims(test_auddf, axis=-1)  # Add a new axis for the channel dimension
        
        # Classify emotion using the model
        predictions = model.predict(test_auddf)
        predicted_class_indices = predictions.argmax(axis=1)
        predicted_emotions = lb.inverse_transform(predicted_class_indices)
        confidence_scores = predictions.max(axis=1)
        all_emotions = lb.classes_  # Get all emotion labels

        for emotion, score, all_scores in zip(predicted_emotions, confidence_scores, predictions):
            score_dict = {emotion: score for emotion, score in zip(all_emotions, all_scores)}
            print(f"Predicted Emotion: {emotion}")
            for emo, sc in score_dict.items():
                print(f"{emo}: {sc:.2f}")
                
                
         # Define file paths for storing history and last prediction
        history_file_path = 'D:\Final Project\Speech\predictions_history.txt'
        last_prediction_file_path = 'D:\Final Project\Text\Final Output\speech_last_prediction.txt'

        # Write predictions history to file
        with open(history_file_path, 'a') as history_file:
            # Iterate over predictions
            for emotion, score, all_scores in zip(predicted_emotions, confidence_scores, predictions):
                score_dict = {emotion: score for emotion, score in zip(all_emotions, all_scores)}

                # Write predicted emotion to the file
                history_file.write(f"Predicted Emotion: {emotion}\n")

                # Write confidence scores for all emotions to the file (excluding 'calm')
                scores_without_calm = [f"{sc:.2f}" for emo, sc in score_dict.items() if emo != 'calm']
                history_file.write(' '.join(scores_without_calm) + '\n')

                # Write a newline to separate predictions
                history_file.write('\n')

        # Write last prediction to file (overwriting previous content)
        with open(last_prediction_file_path, 'w') as last_prediction_file:
            # Get the last prediction
            last_prediction = predicted_emotions[-1]
            last_score_dict = {emotion: score for emotion, score in zip(all_emotions, predictions[-1])}

#             # Write predicted emotion to the file
#             last_prediction_file.write(f"Last Predicted Emotion: {last_prediction}\n")

            # Write confidence scores for all emotions to the file (excluding 'calm')
            last_scores_without_calm = [f"{sc:.2f}" for emo, sc in last_score_dict.items() if emo != 'calm']
            last_prediction_file.write(' '.join(last_scores_without_calm) + '\n')

        # Print message indicating file write completion
        print(f"Predictions history saved to: {history_file_path}")
        print(f"Last prediction saved to: {last_prediction_file_path}")

           
        
except KeyboardInterrupt:
    # Stop recording and close the stream
    print("Recording stopped.")
    stream.stop_stream()
    stream.close()
    audio.terminate()


Recording...
Recording saved as 'audio.wav'
1/1 [==============================] - 0s 182ms/step
Predicted Emotion: disgust
angry: 0.16
calm: 0.14
disgust: 0.25
fear: 0.09
happy: 0.08
neutral: 0.08
sad: 0.09
surprise: 0.11
Predictions history saved to: D:\Final Project\Speech\predictions_history.txt
Last prediction saved to: D:\Final Project\Text\Final Output\speech_last_prediction.txt
Recording...
Recording saved as 'audio.wav'
1/1 [==============================] - 0s 19ms/step
Predicted Emotion: disgust
angry: 0.12
calm: 0.18
disgust: 0.23
fear: 0.10
happy: 0.05
neutral: 0.11
sad: 0.12
surprise: 0.09
Predictions history saved to: D:\Final Project\Speech\predictions_history.txt
Last prediction saved to: D:\Final Project\Text\Final Output\speech_last_prediction.txt
Recording...
Recording saved as 'audio.wav'
1/1 [==============================] - 0s 20ms/step
Predicted Emotion: disgust
angry: 0.15
calm: 0.13
disgust: 0.28
fear: 0.08
happy: 0.09
neutral: 0.07
sad: 0.10
surprise: 0.10
P

In [35]:
# test_aud = extract_mfcc(r'D:\Final Project\Speech\recorded_audio.wav')
# test_aud
# #mean = np.mean(test_aud,axis=0)
# #std = np.std(test_aud,axis=0)
# test_auddf = (test_aud-mean)/std
# test_auddf = np.array([test_auddf])
# test_auddf
# test_auddf = test_auddf[:,:,np.newaxis]
# test_auddf
# # livedf2= pd.DataFrame(data=test_auddf)
# # livedf2 = livedf2.stack().to_frame().T
# # twodim= np.expand_dims(livedf2, axis=2)

In [36]:
# predictions = model.predict(test_auddf, batch_size=1)
# predicted_class_indices = predictions.argmax(axis=1)
# predicted_emotions = lb.inverse_transform(predicted_class_indices)
# confidence_scores = predictions.max(axis=1)
# all_emotions = lb.classes_  # Get all emotion labels

# for emotion, score, all_scores in zip(predicted_emotions, confidence_scores, predictions):
#     score_dict = {emotion: score for emotion, score in zip(all_emotions, all_scores)}
#     print(f"Predicted Emotion: {emotion}")
#     for emo, sc in score_dict.items():
#         print(f"{emo}: {sc:.2f}")


In [37]:
# Define file paths for storing history and last prediction

history_file_path = 'D:\Final Project\Speech\predictions_history.txt'
last_prediction_file_path = 'D:\Final Project\Speech\last_prediction.txt'

# Write predictions history to file

with open(history_file_path, 'a') as history_file:
    
    # Iterate over predictions
    
    for emotion, score, all_scores in zip(predicted_emotions, confidence_scores, predictions):
        score_dict = {emotion: score for emotion, score in zip(all_emotions, all_scores)}
        
        # Write predicted emotion to the file
        
        history_file.write(f"Predicted Emotion: {emotion}\n")
        
        # Write confidence scores for all emotions to the file
        
        for emo, sc in score_dict.items():
            history_file.write(f"{emo}: {sc:.2f}\n")
        
        # Write a newline to separate predictions
        history_file.write('\n')

# Write last prediction to file (overwriting previous content)

with open(last_prediction_file_path, 'w') as last_prediction_file:
    
    # Get the last prediction
    
    last_prediction = predicted_emotions[-1]
    last_score_dict = {emotion: score for emotion, score in zip(all_emotions, predictions[-1])}
    
    # Write predicted emotion to the file
    
    last_prediction_file.write(f"Last Predicted Emotion: {last_prediction}\n")
    
    # Write confidence scores for all emotions to the file
    
    for emo, sc in last_score_dict.items():
        last_prediction_file.write(f"{emo}: {sc:.2f}\n")

# Print message indicating file write completion

print(f"Predictions history saved to: {history_file_path}")
print(f"Last prediction saved to: {last_prediction_file_path}")


Predictions history saved to: D:\Final Project\Speech\predictions_history.txt
Last prediction saved to: D:\Final Project\Speech\last_prediction.txt


In [38]:
# from IPython.display import Audio
# y,sr = librosa.load(r'D:\Final Project\Speech\avatar.wav')
# Audio(data=y,rate=sr)